# Object Following - Live Demo

In this notebook we'll show how you can follow an object with JetBot!  We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices).  The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection),
which provides utilities for training object detectors for custom tasks also!  Once the model is trained, we optimize it using NVIDIA TensorRT on the Jetson Nano.

This makes the network very fast, capable of real-time execution on Jetson Nano!  We won't run through all of the training and optimization steps in this notebook though.

Anyways, let's get started.  First, we'll want to import the ``ObjectDetector`` class which takes our pre-trained SSD engine.


# 오브젝트 따라가기 - 라이브 데모

이 노트북에서는 JetBot로 개체를 따라가는 방법을 보여줍니다! 우리는 미리 훈련 된 신경망을 사용할 것입니다
[COCO dataset] (http://cocodataset.org)에서 90 개의 서로 다른 공통 객체를 감지하도록 훈련했습니다. 이들은 포함합니다

* 사람 (인덱스 0)
* 컵 (색인 47)

그리고 다른 많은 것들 ([이 파일] (https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt)에서 클래스 인덱스의 전체 목록을 확인할 수 있습니다). 모델은 [TensorFlow 객체 감지 API] (https://github.com/tensorflow/models/tree/master/research/object_detection)에서 제공됩니다.
사용자 지정 작업을위한 물체 탐지기 교육용 유틸리티도 제공합니다! 모델 학습이 완료되면 Jetson Nano에서 NVIDIA TensorRT를 사용하여 모델을 최적화합니다.

이로 인해 Jetson Nano에서 네트워크를 매우 빠르게 실행하고 실시간으로 실행할 수 있습니다! 하지만이 노트북에서는 모든 교육 및 최적화 단계를 수행하지는 않습니다.

어쨌든 시작합시다. 먼저 사전 훈련 된 SSD 엔진을 사용하는``ObjectDetector ''클래스를 가져와야합니다.

### Compute detections on single camera image

## 싱글 카메라 이미지에서 Compute 감지


In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

FileNotFoundError: [Errno 2] No such file or directory: 'ssd_mobilenet_v2_coco.engine'

Internally, the ``ObjectDetector`` class uses the TensorRT Python API to execute the engine that we provide.  It also takes care of preprocessing the input to the neural network, as
well as parsing the detected objects.  Right now it will only work for engines created using the ``jetbot.ssd_tensorrt`` package. That package has the utilities for converting
the model from the TensorFlow object detection API to an optimized TensorRT engine.

Next, let's initialize our camera.  Our detector takes 300x300 pixel input, so we'll set this when creating the camera.

> Internally, the Camera class uses GStreamer to take advantage of Jetson Nano's Image Signal Processor (ISP).  This is super fast and offloads
> a lot of the resizing computation from the CPU. 

내부적으로``ObjectDetector ''클래스는 TensorRT Python API를 사용하여 우리가 제공하는 엔진을 실행합니다. 또한 신경망에 대한 입력을 사전 처리합니다.
탐지 된 개체를 파싱 할 수 있습니다. 현재는``jetbot.ssd_tensorrt ''패키지를 사용하여 생성 된 엔진에서만 작동합니다. 이 패키지에는 변환을위한 유틸리티가 있습니다
TensorFlow 객체 감지 API에서 최적화 된 TensorRT 엔진으로의 모델.

다음으로 카메라를 초기화하겠습니다. 검출기는 300x300 픽셀 입력을 사용하므로 카메라를 만들 때이를 설정합니다.

> 내부적으로 Camera 클래스는 GStreamer를 사용하여 Jetson Nano의 이미지 신호 프로세서 (ISP)를 활용합니다. 이것은 매우 빠르고 오프로드입니다.
> CPU에서 많은 크기 조정 계산.

In [ ]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

Now, let's execute our network using some camera input.  By default the ``ObjectDetector`` class expects ``bgr8`` format that the camera produces.  However,
you could override the default pre-processing function if your input is in a different format.

이제 카메라 입력을 사용하여 네트워크를 실행 해 봅시다. 기본적으로``ObjectDetector ''클래스는 카메라가 생성하는``bgr8 ''형식을 필요로합니다. 하나,
입력이 다른 형식 인 경우 기본 전처리 기능을 무시할 수 있습니다.


In [ ]:
detections = model(camera.value)

print(detections)

If there are any COCO objects in the camera's field of view, they should now be stored in the ``detections`` variable.


카메라의 시야에 COCO 객체가있는 경우 이제``detections ''변수에 저장해야합니다.

### Display detections in text area

We'll use the code below to print out the detected objects.


### 텍스트 영역에 감지 표시

아래 코드를 사용하여 감지 된 개체를 인쇄합니다.


In [ ]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

You should see the label, confidence, and bounding box of each object detected in each image.  There's only one image (our camera) in this example. 


To print just the first object detected in the first image, we could call the following

> This may throw an error if no objects are detected



각 이미지에서 감지 된 각 개체의 레이블, 신뢰도 및 경계 상자가 표시되어야합니다. 이 예에는 하나의 이미지 (카메라) 만 있습니다.


첫 번째 이미지에서 감지 된 첫 번째 개체 만 인쇄하려면 다음을 호출하면됩니다.

> 물체가 감지되지 않으면 오류가 발생할 수 있습니다


In [ ]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

### Control robot to follow central object

Now we want our robot to follow an object of the specified class.  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision, this is the 'target' object
3.  Steer robot towards target object, otherwise wander
4.  If we're blocked by an obstacle, turn left

We'll also create some widgets that we'll use to control the target object label, the robot speed, and
a "turn gain", that will control how fast the robot turns based off the distance between the target object
and the center of the robot's field of view. 


First, let's load our collision detection model.  The pre-trained model is stored in this directory as a convenience, but if you followed
the collision avoidance example you may want to use that model if it's better tuned for your robot's environment.



### 중앙 물체를 따르도록 로봇 제어

이제 로봇이 지정된 클래스의 객체를 따르기를 원합니다. 이를 위해 다음을 수행합니다.

1. 지정된 클래스와 일치하는 객체를 감지
2. 카메라의 시야 중심에 가장 가까운 물체를 선택하십시오. 이것은 '대상'물체입니다.
3. 대상을 향해 로봇을 조종하십시오. 그렇지 않으면 방황하십시오.
4. 장애물로 막힌 경우 좌회전

또한 대상 객체 레이블, 로봇 속도 및
대상 물체 사이의 거리에 따라 로봇이 얼마나 빨리 회전하는지 제어하는 ​​"회전 게인"
로봇의 시야 중심.


먼저 충돌 감지 모델을로드하겠습니다. 사전 훈련 된 모델은 편의를 위해이 디렉토리에 저장되지만
충돌 회피 예제 로봇 환경에 더 잘 맞는 모델을 사용하고 싶을 수 있습니다.


In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import cv2
import numpy as np

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)
collision_model.load_state_dict(torch.load('../collision_avoidance/best_model.pth'))
device = torch.device('cuda')
collision_model = collision_model.to(device)

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.resize(x, (224, 224))
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great, now let's initialize our robot so we can control the motors.

좋습니다. 이제 로봇을 초기화하여 모터를 제어 할 수 있습니다.

In [ ]:
from jetbot import Robot

robot = Robot()

Finally, let's display all the control widgets and connect the network execution function to the camera updates.


마지막으로 모든 제어 위젯을 표시하고 네트워크 실행 기능을 카메라 업데이트에 연결하겠습니다.

In [ ]:
from jetbot import bgr8_to_jpeg

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.IntText(value=1, description='tracked label')
speed_widget = widgets.FloatSlider(value=0.4, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.8, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    label_widget,
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
    
def norm(vec):
    """Computes the length of the 2D vector"""
    return np.sqrt(vec[0]**2 + vec[1]**2)

def closest_detection(detections):
    """Finds the detection closest to the image center"""
    closest_detection = None
    for det in detections:
        center = detection_center(det)
        if closest_detection is None:
            closest_detection = det
        elif norm(detection_center(det)) < norm(detection_center(closest_detection)):
            closest_detection = det
    return closest_detection
        
def execute(change):
    image = change['new']
    
    # execute collision model to determine if blocked
    collision_output = collision_model(preprocess(image)).detach().cpu()
    prob_blocked = float(F.softmax(collision_output.flatten(), dim=0)[0])
    blocked_widget.value = prob_blocked
    
    # turn left if blocked
    if prob_blocked > 0.5:
        robot.left(0.3)
        image_widget.value = bgr8_to_jpeg(image)
        return
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    for det in detections[0]:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
    
    # select detections that match selected class label
    matching_detections = [d for d in detections[0] if d['label'] == int(label_widget.value)]
    
    # get detection closest to center of field of view and draw it
    det = closest_detection(matching_detections)
    if det is not None:
        bbox = det['bbox']
        cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (0, 255, 0), 5)
    
    
        
    # otherwise go forward if no target detected
    if det is None:
        robot.forward(float(speed_widget.value))
        
    # otherwsie steer towards target
    else:
        # move robot forward and steer proportional target's x-distance from center
        center = detection_center(det)
        robot.set_motors(
            float(speed_widget.value + turn_gain_widget.value * center[0]),
            float(speed_widget.value - turn_gain_widget.value * center[0])
        )
    
    # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

Call the block below to connect the execute function to each camera frame update.



아래 블록을 호출하여 각 카메라 프레임 업데이트에 실행 기능을 연결하십시오.

In [ ]:
camera.unobserve_all()
camera.observe(execute, names='value')

Awesome!  If the robot is not blocked you should see boxes drawn around the detected objects in blue.  The target object (which the robot follows) will be displayed in green.

The robot should steer towards the target when it is detected.  If it is blocked by an object it will simply turn left.

You can call the code block below to manually disconnect the processing from the camera and stop the robot.


대박! 로봇이 막히지 않으면 감지 된 물체 주위에 파란색 상자가 표시됩니다. 로봇이 따르는 대상 물체는 녹색으로 표시됩니다.

로봇은 물체가 감지 될 때 목표쪽으로 향해야합니다. 물체에 의해 막히면 왼쪽으로 회전합니다.

아래 코드 블록을 호출하여 카메라에서 처리를 수동으로 분리하고 로봇을 중지 할 수 있습니다.

In [ ]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()